In [1]:
import time
import pickle
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
with open(r"..\PickleFiles\trained_ohe.pkl", "rb") as ohe_file:
    trained_ohe = pickle.load(ohe_file)

def transformIP(ip_add):
    if ip_add == 0 or ip_add == '0':
        return 0
    else:
        parts = ip_add.split('.')
        return (int(parts[0]) << 24) + (int(parts[1]) << 16) + (int(parts[2]) << 8) + int(parts[3])
    
def process_db():
    connection = mysql.connector.connect(
    host = "localhost", 
    user = "root", 
    password = "Nad.Safwat123", 
    database = "ml"
    )
    
    df_train = pd.read_sql_query('SELECT * FROM data_to_view', con=connection)
    
    ohetransform = trained_ohe.transform(df_train[['geo_loc', 'tld']]).astype('int8')
    df_train = pd.concat([df_train,ohetransform], axis=1).drop(columns=['geo_loc', 'tld'])

    df_train['ip_add'] = df_train.ip_add.apply(transformIP)

    column = df_train.pop('label')
    df_train[column.name] = column

    df_train = df_train.iloc[:, 1:]

    x_train = df_train[df_train.columns[:-1]]
    y_train = df_train['label']

    return x_train, y_train

def retrain():
    print('Function Opened')

    with open(r'..\PickleFiles\Model.pkl', 'rb') as m:
        model = pickle.load(m)

    x_train, y_train = process_db()

    print('DB Read & Fitting model')
    model.fit(x_train, y_train)

    # overwrite the pickle to save the new training
    with open(r'..\PickleFiles\Model.pkl', 'wb') as m:
        pickle.dump(model, m)
    
    print('Retrained Model')
    return 'Retrained Model'

def get_row_count():
    connection = mysql.connector.connect(
    host = "localhost", 
    user = "root", 
    password = "Nad.Safwat123", 
    database = "ml"
    )
    NumOfRows = pd.read_sql_query('SELECT COUNT(*) AS NumOfRows FROM ml.data_to_view', con=connection)
    return NumOfRows['NumOfRows'].iloc[0]

def monitor_database():
    previous_row_count = get_row_count()
    current_row_count = previous_row_count
    i = 0
    while True:
        current_row_count = get_row_count()# Replace with your actual query
        
        print(f"Iteration {i}")
        print(f"Previous Row Count: {previous_row_count}")
        print(f"Current Row Count: {current_row_count}")
        
        if current_row_count > previous_row_count:
            # Rows increased by 10 or more
            if (current_row_count - previous_row_count) >= 2:
                retrain()  # Call your retrain function here
                previous_row_count = current_row_count
        
        i+= 1
        
        # Sleep for a while before checking again (adjust the interval as needed)
        time.sleep(60)  # Check every minute (you can change this)

In [3]:
monitor_database()

Iteration 0
Previous Row Count: 1200004
Current Row Count: 1200004
Iteration 1
Previous Row Count: 1200004
Current Row Count: 1200004
Iteration 2
Previous Row Count: 1200004
Current Row Count: 1200005
Iteration 3
Previous Row Count: 1200004
Current Row Count: 1200005
Iteration 4
Previous Row Count: 1200004
Current Row Count: 1200005
Iteration 5
Previous Row Count: 1200004
Current Row Count: 1200005
Iteration 6
Previous Row Count: 1200004
Current Row Count: 1200006
Function Opened


KeyboardInterrupt: 